In [ ]:
import pyspark
from pyspark.sql import SparkSession
sc_conf = pyspark.SparkConf()
sc_conf.setAppName("hive-spark-app")
sc_conf.set('spark.executor.cores','1')
sc_conf.set("spark.executor.memory", '1g')
sc_conf.set('hive.metastore.uris', 'thrift://hive-metastore-01:9083')
spark = (SparkSession.builder.config(conf=sc_conf)
         .enableHiveSupport()
         .getOrCreate())

In [ ]:
spark.sql('show tables').show()

In [ ]:
# DROP TABLE text_table;
# CREATE TABLE text_table (key BIGINT, value STRING);
# DESCRIBE FORMATTED text_table;

text_table_name="text_table"

spark.table(text_table_name).show()

In [ ]:
df = spark.read.json("file:/bdpc/data/input/samples/lab05.json")
df.select(df.data.id, df.channel).write.insertInto("text_table")

In [ ]:
# SELECT * FROM text_table LIMIT 5;

text_table_name="text_table"

spark.catalog.refreshTable(text_table_name)
dftext = spark.table(text_table_name)
print(dftext.count())
dftext.show(5)
dftext.explain()

In [ ]:
# CREATE TABLE parquet_table (x BIGINT, y STRING) stored as parquet;
# DESCRIBE FORMATTED parquet_table;

parquet_table_name="parquet_table"

df = spark.read.json("file:/bdpc/data/input/samples/lab05.json")
dfpt = df.select(df.data.id.alias("x"), df.channel.alias("y"))
dfpt.printSchema()
dfpt.show(5)
dfpt.write.format("hive").mode("append").saveAsTable(parquet_table_name)
#dfpt.write.insertInto(parquet_table_name)

In [ ]:
# SELECT * FROM parquet_table LIMIT 5;

parquet_table_name="parquet_table"

spark.catalog.refreshTable(parquet_table_name)
dfp = spark.table(parquet_table_name)
print(dfp.count())
dfp.printSchema()
dfp.show(5)
dfp.explain()